In [1]:
import pandas as pd
import numpy as np

In [2]:
# Read data create columns for tenor
df = pd.read_excel('data/liquidity/CAD_20130223-20130310.xlsx')
df['Tenor'] = np.round((df['Maturity'] - df['Effective']).dt.days / 365.25)
df['Trade Date'] = df['Trade Time'].dt.date


In [3]:
def calculate_roll_measure(prices):
    """
    Calculate the Roll measure from a list of trade prices.
    
    Parameters:
    prices (list or numpy array): A list or numpy array of trade prices.
    
    Returns:
    float: The Roll measure in decimal form.
    """    
    # Calculate the price changes (Delta P_t)
    delta_P_t = np.diff(prices)
    
    # Calculate the covariance between delta_P_t and delta_P_t_minus_1
    delta_P_t_minus_1 = delta_P_t[:-1]
    delta_P_t = delta_P_t[1:]
    
    cov = np.cov(delta_P_t, delta_P_t_minus_1, ddof=0)[0, 1]
    
    # Calculate the roll measure
    roll_measure = 2 * np.sqrt(-cov)
    
    # Convert the roll measure to basis points
    roll_measure_bps = roll_measure * 10000
    
    return roll_measure, roll_measure_bps


In [4]:
def apply_roll_measure(group):
    # Extract the prices as a numpy array
    prices = group['Rate 1'].values
    # Calculate the roll measure
    roll_measure, roll_measure_bps = calculate_roll_measure(prices)
    return pd.Series({'Roll Measure': roll_measure, 'Roll Measure (bps)': roll_measure_bps})

In [5]:
filtered_df = df[(df['CD'] == 'TR') & (df['Tenor'] == 10.0) & (df["Leg 2"]== 'USD-LIBOR-BBA')]
filtered_df = filtered_df.sort_values('Trade Time')
# result = filtered_df.groupby('Trade Date').apply(apply_roll_measure).reset_index()

/var/folders/v2/2g59sj7s7qbcgl_0l5zkx1gm0000gn/T/ipykernel_55058/2249901530.py:3: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  result = filtered_df.groupby('Trade Date').apply(apply_roll_measure).reset_index()


ValueError: cannot insert Trade Date, already exists

In [25]:
result.to_csv('data/liquidity/Roll/cad_roll_phase_1.csv')